# Feature Barcode pre-processing with kite

This notebook shows how to use kallisto | bustools for fast and accurate pre-processing for Feature Barcoding experiments, a common datatype in single-cell genomics. It uses kallisto 0.46 and bustools 0.39.2. ScanPy is used for downstream analysis. 

In Feature Barcoding assays, cellular data are recorded as short DNA sequences using procedures adapted from single-cell RNA-seq. The __kITE ("kallisto Indexing and Tag Extraction__") workflow involves generating a "Mismatch Map" containing the sequences of all Feature Barcodes used in the experiment as well as all of their single-base mismatches. The Mismatch Map is used to produce transcipt-to-gene (t2g) and fasta files to be used as inputs for kallisto. An index is made with `kallisto index`, then kallisto | bustools  effectively searches the sequencing data for the sequences in the Mismatch Map. We find that for Feature Barcodes of moderate length (6-15bp) pre-processing is remarkably fast and the results equivalent to or better than those from traditional alignment. 

This notebook contains python scripts used to generate the required t2g and fasta files starting with a list of __Feature Barcodes in a Python dictionary object__. To illustrate their use, data from the 10x Genomics __pbmc_1k_protein_v3__ dataset were used and the results compared with CellRanger. 

### Install and test kite

Use the Terminal to __Navigate to a new working directory__ and __download kite from GitHub__

```
mkdir ./10xTest/
cd ./10xTest/
git clone https://github.com/jgehringUCB/kite 
mv ./kite/docs/10x_kiteVignette.ipynb ./
```

Move this notebook from the `./kite/docs/` folder to the working directory `./`, then open it and begin...

In [1]:
!ls

10x_kiteVignette.ipynb	kite


In [2]:
!kallisto version

kallisto, version 0.46.0


In [3]:
!bustools

bustools 0.39.2

Usage: bustools <CMD> [arguments] ..

Where <CMD> can be one of: 

sort            Sort bus file by barcodes and UMI
text            Output as tab separated text file
correct         Error correct bus files
count           Generate count matrices from bus file
capture         Capture reads mapping to a transcript capture list

Running bustools <CMD> without arguments prints usage information for <CMD>



In [4]:
!pip install --force-reinstall -e ./kite

Obtaining file:///home/jgehring/scRNAseq/kITE/10xTest2/kite
  Found existing installation: kite 0.0.1
    Uninstalling kite-0.0.1:
      Successfully uninstalled kite-0.0.1
  Running setup.py develop for kite


In [5]:
import kite

In [6]:
kite.version()

0.0.1


### Download Dataset - raw fastqs

Link to data download page: [10xPBMC_1k_protein_v3](https://support.10xgenomics.com/single-cell-gene-expression/datasets/3.0.0/pbmc_1k_protein_v3)

In [7]:
!wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_fastqs.tar
!tar -xvf ./pbmc_1k_protein_v3_fastqs.tar

--2019-06-28 10:47:18--  http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_fastqs.tar
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 13.35.99.77, 13.35.99.113, 13.35.99.80, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|13.35.99.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4658677760 (4.3G) [application/x-tar]
Saving to: ‘pbmc_1k_protein_v3_fastqs.tar’

100%[====================================>] 4,658,677,760  112MB/s   in 41s    

2019-06-28 10:47:59 (109 MB/s) - ‘pbmc_1k_protein_v3_fastqs.tar’ saved [4658677760/4658677760]

pbmc_1k_protein_v3_fastqs/
pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/
pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L002_R2_001.fastq.gz
pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L001_I1_001.fastq.gz
pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1

We should also grab the antibody barcode sequences which 10x Genomics has placed in a csv file. 

In [8]:
!wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_feature_ref.csv

--2019-06-28 10:48:04--  http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_feature_ref.csv
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 13.35.99.77, 13.35.99.80, 13.35.99.113, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|13.35.99.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1473 (1.4K) [text/csv]
Saving to: ‘pbmc_1k_protein_v3_feature_ref.csv’

100%[======================================>] 1,473       --.-K/s   in 0s      

2019-06-28 10:48:04 (115 MB/s) - ‘pbmc_1k_protein_v3_feature_ref.csv’ saved [1473/1473]



And I'll copy the CellRanger cell barcode whitelist here as well. You can download it as part of CellRanger. See this link https://kb.10xgenomics.com/hc/en-us/articles/115004506263-What-is-a-barcode-whitelist-

In [9]:
!mv ./kite/docs/3M-february-2018.txt.gz ./
!gunzip ./3M-february-2018.txt.gz

mv: cannot stat ‘./kite/docs/3M-february-2018.txt.gz’: No such file or directory
gzip: ./3M-february-2018.txt.gz: No such file or directory


In [10]:
!ls

10x_kiteVignette.ipynb	   pbmc_1k_protein_v3_fastqs.tar
kite			   pbmc_1k_protein_v3_feature_ref.csv
pbmc_1k_protein_v3_fastqs


### Prepare a fasta file of Feature Barcodes

Create a fasta file containing only the Feature Barcode sequences (no common or constant sequences) and corresponding Feature Names used in the experiment. Place this file in the same directory as the raw fastq files. In this case, 10x has provided a csv file that I parse using the get_tags function below. The objects tags is the Python Feature Barocde dictionary that I need for kite. 

In [11]:
import csv

In [12]:
def get_tags(filename):
    with open(filename, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        tags = {}
        next(csv_reader)
        for row in csv_reader:
            tags[row[1].strip()] = row[4].strip()
    return tags

tags = get_tags('./pbmc_1k_protein_v3_feature_ref.csv')

There are 17 Feature Barcodes in this experiment.

In [13]:
tags

{'CD3_TotalSeqB': 'AACAAGACCCTTGAG',
 'CD4_TotalSeqB': 'TACCCGTAATAGCGT',
 'CD8a_TotalSeqB': 'ATTGGCACTCAGATG',
 'CD14_TotalSeqB': 'GAAAGTCAAAGCACT',
 'CD15_TotalSeqB': 'ACGAATCAATCTGTG',
 'CD16_TotalSeqB': 'GTCTTTGTCAGTGCA',
 'CD56_TotalSeqB': 'GTTGTCCGACAATAC',
 'CD19_TotalSeqB': 'TCAACGCTTGGCTAG',
 'CD25_TotalSeqB': 'GTGCATTCAACAGTA',
 'CD45RA_TotalSeqB': 'GATGAGAACAGGTTT',
 'CD45RO_TotalSeqB': 'TGCATGTCATCGGTG',
 'PD-1_TotalSeqB': 'AAGTCGTGAGGCATG',
 'TIGIT_TotalSeqB': 'TGAAGGCTCATTTGT',
 'CD127_TotalSeqB': 'ACATTGACGCAACTA',
 'IgG2a_control_TotalSeqB': 'CTCTATTCAGACCAG',
 'IgG1_control_TotalSeqB': 'ACTCACTGGAGTCTC',
 'IgG2b_control_TotalSeqB': 'ATCACATCGTTGCCA'}

### Prepare mismatch fasta and t2g files

Now call the kITE_Mismatch_Maps function with a fasta file of whitelist Feature Barcodes and destination paths for a newly generated mismatch t2g and mismatch fasta. 

In [14]:
"t2g_path is where your t2g file will be placed. This file is used by the bustools count function"
t2g_path = "./10xFeatures_t2g.txt"

"fasta_path is where your fasta file will be placed. This file is used by the kallisto function"
fasta_path= "./10xFeaturesMismatch.fa"

kite.kite_mismatch_maps(tags, t2g_path, fasta_path)

Feature Barcode Length: 15

Read the following Feature Barcodes:
CD3_TotalSeqB
AACAAGACCCTTGAG
CD4_TotalSeqB
TACCCGTAATAGCGT
CD8a_TotalSeqB
ATTGGCACTCAGATG
CD14_TotalSeqB
GAAAGTCAAAGCACT
CD15_TotalSeqB
ACGAATCAATCTGTG
CD16_TotalSeqB
GTCTTTGTCAGTGCA
CD56_TotalSeqB
GTTGTCCGACAATAC
CD19_TotalSeqB
TCAACGCTTGGCTAG
CD25_TotalSeqB
GTGCATTCAACAGTA
CD45RA_TotalSeqB
GATGAGAACAGGTTT
CD45RO_TotalSeqB
TGCATGTCATCGGTG
PD-1_TotalSeqB
AAGTCGTGAGGCATG
TIGIT_TotalSeqB
TGAAGGCTCATTTGT
CD127_TotalSeqB
ACATTGACGCAACTA
IgG2a_control_TotalSeqB
CTCTATTCAGACCAG
IgG1_control_TotalSeqB
ACTCACTGGAGTCTC
IgG2b_control_TotalSeqB
ATCACATCGTTGCCA
The t2g and fasta files are now ready


In [15]:
!ls '../10xTest/'

10xFeaturesMismatch.fa	    output.bus
10xFeaturesMismatch.idx     output_corrected.bus
10xFeatures_t2g.txt	    output_sorted.bus
10x_kiteVignette.ipynb	    pbmc_1k_protein_v3_fastqs
3M-february-2018.txt	    pbmc_1k_protein_v3_fastqs.tar
filtered_feature_bc_matrix  pbmc_1k_protein_v3_feature_ref.csv
genecount		    run_info.json
kite			    transcripts.txt
matrix.ec


# From here, kallisto | bustools is followed in a standard analysis

With the files produced above, the kallisto single-cell pipeline is employed using the mismatch fasta and t2g files generated above. In kallisto index, the mismatch fasta is used along with a k-mer length equal the length of the Feature Barcode. In bustools count, the mismatch t2g file is referenced. 


    
The rest of this notebook completes the analysis using kallisto | bustools and compares the results to CellRanger using the ScanPy single-cell analysis package. 

In [16]:
"""
Generate a kallisto index, setting the k-mer length k to the Feature Barcode length.
"""

!kallisto index -i ./10xFeaturesMismatch.idx -k 15 ./10xFeaturesMismatch.fa


[build] loading fasta file ./10xFeaturesMismatch.fa
[build] k-mer length: 15
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 782 contigs and contains 782 k-mers 



In [17]:
"""
Inspect the index
"""

!kallisto inspect ./10xFeaturesMismatch.idx

[index] k-mer length: 15
[index] number of targets: 782
[index] number of k-mers: 782
[index] number of equivalence classes: 782
#[inspect] Index version number = 10
#[inspect] k = 15
#[inspect] number of targets = 782
#[inspect] number of equivalence classes = 782
#[inspect] number of contigs = 782
#[inspect] Number of k-mers in index = 782
#EC.size	Num.targets
1	782


#EC.size	Num.kmers
1	782


In [19]:
"""
Call kallisto bus using the directory of the desired fastq files, the index to be used,
sequencing technology used, number of threads, and output folder
"""

!kallisto bus -i ./10xFeaturesMismatch.idx -o ./ -x 10xv3 -t 4 \
./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L001_R1_001.fastq.gz \
./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L001_R2_001.fastq.gz \
./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L002_R1_001.fastq.gz \
./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L002_R2_001.fastq.gz \



[index] k-mer length: 15
[index] number of targets: 782
[index] number of k-mers: 782
[index] number of equivalence classes: 782
[quant] will process sample 1: ./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L001_R1_001.fastq.gz
                               ./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L001_R2_001.fastq.gz
[quant] will process sample 2: ./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L002_R1_001.fastq.gz
                               ./pbmc_1k_protein_v3_fastqs/pbmc_1k_protein_v3_antibody_fastqs/pbmc_1k_protein_v3_antibody_S2_L002_R2_001.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 12,606,650 reads, 11,517,927 reads pseudoaligned


In [20]:
"""
Call bustools correct to error-correct the barcodes
"""

!bustools correct -w ./3M-february-2018.txt ./output.bus -o ./output_corrected.bus


Error: File not found ./3M-february-2018.txt
Usage: bustools text [options] bus-files

Options: 
-o, --output          File for text output
-p, --pipe            Write to standard output



In [21]:
"""
Call bustools sort to sort the BUS file
"""

!bustools sort -t 4 -o ./output_sorted.bus ./output_corrected.bus

Error: File not found, ./output_corrected.bus
Usage: bustools sort [options] bus-files

Options: 
-t, --threads         Number of threads to use
-m, --memory          Maximum memory used
-T, --temp            Location and prefix for temporary files 
                      required if using -p, otherwise defaults to output
-o, --output          File for sorted output
-p, --pipe            Write to standard output



In [22]:
"""
Call bustools count to generate an error-corrected genes x cells matrix
You will need the t2g file generated earlier as well as standard BUS outputs
"""

!mkdir ./genecount/

!bustools count -o ./genecount/genecounts --genecounts -g ./10xFeatures_t2g.txt -e ./matrix.ec -t ./transcripts.txt ./output_sorted.bus


Error: File not found, ./output_sorted.bus
Usage: bustools text [options] bus-files

Options: 
-o, --output          File for text output
-p, --pipe            Write to standard output



In [23]:
!ls

10xFeaturesMismatch.fa	 output.bus
10xFeaturesMismatch.idx  pbmc_1k_protein_v3_fastqs
10xFeatures_t2g.txt	 pbmc_1k_protein_v3_fastqs.tar
10x_kiteVignette.ipynb	 pbmc_1k_protein_v3_feature_ref.csv
genecount		 run_info.json
kite			 transcripts.txt
matrix.ec


The pre-processing is done, and the data can now be analyzed using, for example, the ScanPy single-cell analysis package. 

In [24]:
import scanpy.api as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

genes=sc.read_mtx('./genecount/genecounts.mtx')

FileNotFoundError: [Errno 2] No such file or directory: './genecount/genecounts.mtx'

In [ ]:
genes

In [ ]:
genes.var_names=pd.read_csv('/home/jgehring/scRNAseq/kITE/10xTest/genecount/genecounts.genes.txt', header=None)[0]
genes.obs_names=pd.read_csv('/home/jgehring/scRNAseq/kITE/10xTest/genecount/genecounts.barcodes.txt', header=None)[0]


In [ ]:
genes.var_names

In [ ]:
print(sum(genes.X))

In [ ]:
sc.pp.filter_cells(genes,min_counts=0)

In [ ]:
sc.pp.filter_genes(genes,min_counts=0)

In [ ]:
sc.pl.violin(genes, keys='n_counts')

In [ ]:
genes.obs['n_countslog']=np.log1p(genes.obs['n_counts'])

In [ ]:
sc.pl.violin(genes, keys='n_countslog')

In [ ]:
genes.obs_names

In [ ]:
sc.pp.filter_cells(genes, min_counts=1000)
sc.pl.violin(genes, keys='n_countslog', title="kallisto UMI counts")
genes

In [ ]:
sc.pp.normalize_per_cell(genes, counts_per_cell_after=10000)

In [ ]:
sc.pp.neighbors(genes)

sc.tl.umap(genes)

In [ ]:
sc.tl.leiden(genes, resolution=0.05)

In [ ]:
sc.pl.umap(genes, color='leiden', palette='tab10')

We can download the 10x Feature Barcodes x Cells matrix for comparison

In [ ]:
!wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_filtered_feature_bc_matrix.tar.gz -P ../

In [ ]:
!tar xvzf ../pbmc_1k_protein_v3_filtered_feature_bc_matrix.tar.gz

In [ ]:
!ls ../filtered_feature_bc_matrix/

In [ ]:
tenx = sc.read_mtx('../filtered_feature_bc_matrix/matrix.mtx.gz').T

In [ ]:

tenx.var_names=pd.read_csv('../filtered_feature_bc_matrix/features.tsv.gz', delimiter='\t', header=None)[1]



In [ ]:
tenx.var_names_make_unique()

In [ ]:
tenx

In [ ]:
list(tenx.var_names[-17:])

In [ ]:
tenxT=tenx.copy().T
tenx=tenxT[tenxT.obs_names.isin(list(tenx.var_names[-17:]))].copy().T

In [ ]:
tenx

In [ ]:
tenx.var_names

In [ ]:
sc.pp.filter_cells(tenx, min_counts=0)
tenx.obs['n_countslog']=np.log1p(tenx.obs['n_counts'])
sc.pl.violin(tenx, keys='n_countslog', title="CellRanger UMI distribution")
tenx

Looks like 10x allowed some low-count cells that we filtered out. Compared with the same violin plot the kallisto analysis ("genes", above), the distributions are very similar.

In [ ]:
sc.pp.normalize_per_cell(tenx, counts_per_cell_after=10000)

sc.pp.neighbors(tenx)

sc.tl.umap(tenx)

sc.tl.leiden(tenx, resolution=0.1)

sc.pl.umap(tenx, color='leiden', palette='tab10')

In [ ]:
sc.pl.violin(tenx, keys=list(tenx.var_names)[-17:], xlabel='CellRanger')
sc.pl.violin(genes, keys=list(genes.var_names)[-17:], xlabel='kallisto')

Here are violin plots for each protein (the Feature Barcodes, x-axis) across all cells. Aagin, kallisto and CellRanger are strikingly similar. 

In [ ]:
genes.var_names

In [ ]:
sc.pl.umap(genes, color=genes.var_names)
print("Embedding and Antibody Quantification using kallisto")

In [ ]:
sc.pl.umap(tenx, color=genes.var_names)
print("Embedding and Antibody Quantification using CellRanger")

In [ ]:
sc.pl.violin(tenx, keys=tenx.var_names[:2], groupby='leiden', title='CellRanger')
print("10x CellRanger")
sc.pl.violin(genes, keys=tenx.var_names[:2], groupby='leiden')
print("kallisto | bustools")

In the plot above, independent analyses by CellRanger and kallisto are compared. The top two violin plots are for the proteins CD3 and CD4 across clusters. Using four clusters for the kallisto analysis gave highly similar expression patterns for the same genes, indicating that the analyses are highly correlated. 

In [ ]:
plt.plot(sum(tenx.X).todense().tolist()[0], sum(tenx.X).todense().tolist()[0])
plt.scatter(sum(tenx.X).todense().tolist()[0], sum(genes.X).todense().tolist()[0], color='r')
plt.ylabel("kallisto")
plt.xlabel("CellRanger")
plt.show()

In the plot above, 'pseudobulk' expression values are compared for each antibody-oligo conjugate (Feature Barcode) using kallisto or CellRanger. The two analyses show a high degree of similarity, with r^2=0.9985

In [ ]:
from scipy import stats
import numpy as np

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(sum(tenx.X).todense().tolist()[0],sum(genes.X).todense().tolist()[0])

In [ ]:
print("r-squared:", r_value**2)